In [13]:
import re
from glob import glob
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd

import torch

from matplotlib import pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, BertModel

pd.set_option("display.max_rows", 300)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

## noise analysis

In [23]:
trn_df = pd.read_csv('../inputs/origin/train.csv').dropna()
display(trn_df.shape, trn_df.head())

(27480, 4)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [24]:
ckpts = []
!mkdir -p ../checkpoints/e152/best_model_only/
for fckpt in glob('../checkpoints/e152/best/*.pth'):
    ckpt = torch.load(fckpt)
    ckpts.append(ckpt)
    fout_ckpt = {}
    fout_ckpt['model_state_dict'] = ckpt['model_state_dict']
    torch.save(fout_ckpt, re.sub('best', 'best_model_only', fckpt))

In [25]:
def _mk_char_preds(offsets, preds_head, preds_tail):
    char_preds_heads, char_preds_tails = [], []
    # 最初の４つは無視
    for offset, pred_head, pred_tail in tqdm(zip(offsets, preds_head, preds_tail)):
        char_preds_head, char_preds_tail = np.zeros(141), np.zeros(141)
        for offset_i, pred_head_i, pred_tail_i in zip(offset[4:], pred_head[4:], pred_tail[4:]):
            char_preds_head[offset_i[0]:offset_i[1]] = pred_head_i
            char_preds_tail[offset_i[0]:offset_i[1]] = pred_tail_i
        char_preds_heads.append(char_preds_head)
        char_preds_tails.append(char_preds_tail)
    char_preds_heads, char_preds_tails = np.asarray(char_preds_heads), np.asarray(char_preds_tails)
    return char_preds_heads, char_preds_tails

In [26]:
ckpts[0].keys()

dict_keys(['fold_num', 'current_epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'val_textIDs', 'val_input_ids', 'val_preds', 'val_labels', 'val_offsets', 'histories'])

In [27]:
char_preds = []

for ckpt in ckpts:
    trn_df = trn_df.set_index('textID')
    val_char_preds_heads, val_char_preds_tails = _mk_char_preds(ckpt['val_offsets'], ckpt['val_preds'][0],  ckpt['val_preds'][1])
    for textID, val_char_preds_head, val_char_preds_tail in zip(ckpt['val_textIDs'], val_char_preds_heads, val_char_preds_tails):
        char_preds.append({'textID': textID, 'char_pred_start': val_char_preds_head, 'char_pred_end': val_char_preds_tail})
    # trn_df.loc[ckpt['val_textIDs'], 'val_char_pred_head'] = val_char_preds_heads
    # trn_df.loc[ckpt['val_textIDs'], 'val_char_pred_tail'] = val_char_preds_tails
    trn_df = trn_df.reset_index()
trn_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [28]:
char_preds_df = pd.DataFrame(char_preds)
char_preds_df.head()

,textID,char_pred_start,char_pred_end
0,cb774db0d1,"[0.9993091821670532, 0.9993091821670532, 0.000...","[0.00010164044942939654, 0.0001016404494293965..."
1,549e992a42,"[0.5705369114875793, 0.5705369114875793, 0.570...","[0.00019916410383302718, 0.0001991641038330271..."
2,088c60f138,"[0.44564589858055115, 0.44564589858055115, 0.4...","[0.00031484506325796247, 0.0003148450632579624..."
3,9642c003ef,"[0.1700505018234253, 0.1700505018234253, 0.170...","[0.00022137271298561245, 0.0002213727129856124..."
4,358bd9e861,"[0.7032945156097412, 0.7032945156097412, 0.703...","[0.0006509069353342056, 0.0006509069353342056,..."


In [29]:
trn_df = trn_df.merge(char_preds_df, on='textID', how='left')
trn_df

,textID,text,selected_text,sentiment,char_pred_start,char_pred_end
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"[0.9993091821670532, 0.9993091821670532, 0.000...","[0.00010164044942939654, 0.0001016404494293965..."
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0.5705369114875793, 0.5705369114875793, 0.570...","[0.00019916410383302718, 0.0001991641038330271..."
2,088c60f138,my boss is bullying me...,bullying me,negative,"[0.44564589858055115, 0.44564589858055115, 0.4...","[0.00031484506325796247, 0.0003148450632579624..."
3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0.1700505018234253, 0.1700505018234253, 0.170...","[0.00022137271298561245, 0.0002213727129856124..."
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[0.7032945156097412, 0.7032945156097412, 0.703...","[0.0006509069353342056, 0.0006509069353342056,..."
...,...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,"[0.03742819279432297, 0.03742819279432297, 0.0...","[0.00019678355602081865, 0.0001967835560208186..."
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,"[0.30980750918388367, 0.30980750918388367, 0.0...","[0.00034203799441456795, 0.0003420379944145679..."
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,"[0.5343130826950073, 0.5343130826950073, 0.003...","[0.0015588045353069901, 0.0015588045353069901,..."
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,"[0.19598905742168427, 0.19598905742168427, 0.1...","[0.0006552892737090588, 0.0006552892737090588,..."


In [30]:
trn_df.isnull().mean()

textID             0.0
text               0.0
selected_text      0.0
sentiment          0.0
char_pred_start    0.0
char_pred_end      0.0
dtype: float64

In [31]:
!mkdir -p ../inputs/datasets/char_preds/e152

In [32]:
trn_df[['textID', 'char_pred_start', 'char_pred_end']].to_pickle('../inputs/datasets/char_preds/e152/e152_char_preds.pkl')

## e153 も

## noise analysis

In [33]:
trn_df = pd.read_csv('../inputs/origin/train.csv').dropna()
display(trn_df.shape, trn_df.head())

(27480, 4)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [34]:
ckpts = []
!mkdir -p ../checkpoints/e153/best_model_only/
for fckpt in glob('../checkpoints/e153/best/*.pth'):
    ckpt = torch.load(fckpt)
    ckpts.append(ckpt)
    fout_ckpt = {}
    fout_ckpt['model_state_dict'] = ckpt['model_state_dict']
    torch.save(fout_ckpt, re.sub('best', 'best_model_only', fckpt))

In [35]:
def _mk_char_preds(offsets, preds_head, preds_tail):
    char_preds_heads, char_preds_tails = [], []
    # 最初の４つは無視
    for offset, pred_head, pred_tail in tqdm(zip(offsets, preds_head, preds_tail)):
        char_preds_head, char_preds_tail = np.zeros(141), np.zeros(141)
        for offset_i, pred_head_i, pred_tail_i in zip(offset[4:], pred_head[4:], pred_tail[4:]):
            char_preds_head[offset_i[0]:offset_i[1]] = pred_head_i
            char_preds_tail[offset_i[0]:offset_i[1]] = pred_tail_i
        char_preds_heads.append(char_preds_head)
        char_preds_tails.append(char_preds_tail)
    char_preds_heads, char_preds_tails = np.asarray(char_preds_heads), np.asarray(char_preds_tails)
    return char_preds_heads, char_preds_tails

In [36]:
ckpts[0].keys()

dict_keys(['fold_num', 'current_epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'val_textIDs', 'val_input_ids', 'val_preds', 'val_labels', 'val_offsets', 'histories'])

In [37]:
char_preds = []

for ckpt in ckpts:
    trn_df = trn_df.set_index('textID')
    val_char_preds_heads, val_char_preds_tails = _mk_char_preds(ckpt['val_offsets'], ckpt['val_preds'][0],  ckpt['val_preds'][1])
    for textID, val_char_preds_head, val_char_preds_tail in zip(ckpt['val_textIDs'], val_char_preds_heads, val_char_preds_tails):
        char_preds.append({'textID': textID, 'char_pred_start': val_char_preds_head, 'char_pred_end': val_char_preds_tail})
    # trn_df.loc[ckpt['val_textIDs'], 'val_char_pred_head'] = val_char_preds_heads
    # trn_df.loc[ckpt['val_textIDs'], 'val_char_pred_tail'] = val_char_preds_tails
    trn_df = trn_df.reset_index()
trn_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [38]:
char_preds_df = pd.DataFrame(char_preds)
char_preds_df.head()

,textID,char_pred_start,char_pred_end
0,863edfea8b,"[0.3847193419933319, 0.3847193419933319, 0.384...","[0.0691322460770607, 0.0691322460770607, 0.069..."
1,a2e1121aa7,"[0.011290576308965683, 0.011290576308965683, 0...","[0.00036885947338305414, 0.0003688594733830541..."
2,4d3aca8926,"[0.9986562728881836, 0.9986562728881836, 0.998...","[0.00012245692778378725, 0.0001224569277837872..."
3,88a400760a,"[0.2519451975822449, 0.2519451975822449, 0.251...","[0.0004826546064577997, 0.0004826546064577997,..."
4,bbbd337b0b,"[0.4297378957271576, 0.4297378957271576, 0.429...","[0.056389953941106796, 0.056389953941106796, 0..."


In [39]:
trn_df = trn_df.merge(char_preds_df, on='textID', how='left')
trn_df

,textID,text,selected_text,sentiment,char_pred_start,char_pred_end
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"[0.9987541437149048, 0.9987541437149048, 0.000...","[3.688297510962002e-05, 3.688297510962002e-05,..."
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0.615781307220459, 0.615781307220459, 0.61578...","[0.00027660999330691993, 0.0002766099933069199..."
2,088c60f138,my boss is bullying me...,bullying me,negative,"[0.3109663724899292, 0.3109663724899292, 0.310...","[0.0013144471449777484, 0.0013144471449777484,..."
3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0.2529185712337494, 0.2529185712337494, 0.252...","[0.0008793757879175246, 0.0008793757879175246,..."
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[0.5312957763671875, 0.5312957763671875, 0.531...","[0.0011636412236839533, 0.0011636412236839533,..."
...,...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,"[0.07795494794845581, 0.07795494794845581, 0.0...","[0.0013425374636426568, 0.0013425374636426568,..."
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,"[0.24684298038482666, 0.24684298038482666, 0.0...","[0.00044112512841820717, 0.0004411251284182071..."
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,"[0.29858100414276123, 0.29858100414276123, 0.0...","[0.0009772243211045861, 0.0009772243211045861,..."
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,"[0.11491180956363678, 0.11491180956363678, 0.1...","[0.0008585219620727003, 0.0008585219620727003,..."


In [40]:
trn_df.isnull().mean()

textID             0.0
text               0.0
selected_text      0.0
sentiment          0.0
char_pred_start    0.0
char_pred_end      0.0
dtype: float64

In [41]:
!mkdir -p ../inputs/datasets/char_preds/e153

In [42]:
trn_df[['textID', 'char_pred_start', 'char_pred_end']].to_pickle('../inputs/datasets/char_preds/e153/e153_char_preds.pkl')

## e154

## noise analysis

In [50]:
trn_df = pd.read_csv('../inputs/origin/train.csv').dropna()
display(trn_df.shape, trn_df.head())

(27480, 4)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [51]:
ckpts = []
!mkdir -p ../checkpoints/e154/best_model_only/
for fckpt in glob('../checkpoints/e154/best/*.pth'):
    ckpt = torch.load(fckpt)
    ckpts.append(ckpt)
    fout_ckpt = {}
    fout_ckpt['model_state_dict'] = ckpt['model_state_dict']
    torch.save(fout_ckpt, re.sub('best', 'best_model_only', fckpt))

In [52]:
def _mk_char_preds(offsets, preds_head, preds_tail):
    char_preds_heads, char_preds_tails = [], []
    # 最初の４つは無視
    for offset, pred_head, pred_tail in tqdm(zip(offsets, preds_head, preds_tail)):
        char_preds_head, char_preds_tail = np.zeros(141), np.zeros(141)
        for offset_i, pred_head_i, pred_tail_i in zip(offset[4:], pred_head[4:], pred_tail[4:]):
            char_preds_head[offset_i[0]:offset_i[1]] = pred_head_i
            char_preds_tail[offset_i[0]:offset_i[1]] = pred_tail_i
        char_preds_heads.append(char_preds_head)
        char_preds_tails.append(char_preds_tail)
    char_preds_heads, char_preds_tails = np.asarray(char_preds_heads), np.asarray(char_preds_tails)
    return char_preds_heads, char_preds_tails

In [53]:
ckpts[0].keys()

dict_keys(['fold_num', 'current_epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'val_textIDs', 'val_input_ids', 'val_preds', 'val_labels', 'val_offsets', 'histories'])

In [54]:
char_preds = []

for ckpt in ckpts:
    trn_df = trn_df.set_index('textID')
    val_char_preds_heads, val_char_preds_tails = _mk_char_preds(ckpt['val_offsets'], ckpt['val_preds'][0],  ckpt['val_preds'][1])
    for textID, val_char_preds_head, val_char_preds_tail in zip(ckpt['val_textIDs'], val_char_preds_heads, val_char_preds_tails):
        char_preds.append({'textID': textID, 'char_pred_start': val_char_preds_head, 'char_pred_end': val_char_preds_tail})
    # trn_df.loc[ckpt['val_textIDs'], 'val_char_pred_head'] = val_char_preds_heads
    # trn_df.loc[ckpt['val_textIDs'], 'val_char_pred_tail'] = val_char_preds_tails
    trn_df = trn_df.reset_index()
trn_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [55]:
char_preds_df = pd.DataFrame(char_preds)
char_preds_df.head()

,textID,char_pred_start,char_pred_end
0,2d43263be0,"[0.89841628074646, 0.89841628074646, 0.8984162...","[0.0003774258366320282, 0.0003774258366320282,..."
1,c554ea2a90,"[0.03897930309176445, 0.03897930309176445, 0.0...","[0.0004395379510242492, 0.0004395379510242492,..."
2,ab039258bd,"[0.04385346546769142, 0.04385346546769142, 0.0...","[0.00030158201116137207, 0.0003015820111613720..."
3,5a85ab5cf9,"[0.9975095987319946, 0.9975095987319946, 0.997...","[0.0005380607326515019, 0.0005380607326515019,..."
4,36f947146a,"[0.10469190776348114, 0.10469190776348114, 0.1...","[0.002065591514110565, 0.002065591514110565, 0..."


In [56]:
trn_df = trn_df.merge(char_preds_df, on='textID', how='left')
trn_df

,textID,text,selected_text,sentiment,char_pred_start,char_pred_end
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"[0.9982798099517822, 0.9982798099517822, 0.000...","[0.00024153896083589643, 0.0002415389608358964..."
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0.3679257035255432, 0.3679257035255432, 0.367...","[0.0004281910660210997, 0.0004281910660210997,..."
2,088c60f138,my boss is bullying me...,bullying me,negative,"[0.3835388720035553, 0.3835388720035553, 0.383...","[0.0008378780912607908, 0.0008378780912607908,..."
3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0.12908340990543365, 0.12908340990543365, 0.1...","[0.0006538929301314056, 0.0006538929301314056,..."
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[0.544698178768158, 0.544698178768158, 0.54469...","[0.0010821548057720065, 0.0010821548057720065,..."
...,...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,"[0.051752399653196335, 0.051752399653196335, 0...","[0.0006701280362904072, 0.0006701280362904072,..."
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,"[0.3703768849372864, 0.3703768849372864, 0.000...","[0.0004222270508762449, 0.0004222270508762449,..."
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,"[0.3994556963443756, 0.3994556963443756, 0.002...","[0.0005538389668799937, 0.0005538389668799937,..."
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,"[0.12966763973236084, 0.12966763973236084, 0.1...","[0.0003283437108621001, 0.0003283437108621001,..."


In [57]:
trn_df.isnull().mean()

textID             0.0
text               0.0
selected_text      0.0
sentiment          0.0
char_pred_start    0.0
char_pred_end      0.0
dtype: float64

In [58]:
!mkdir -p ../inputs/datasets/char_preds/e154

In [59]:
trn_df[['textID', 'char_pred_start', 'char_pred_end']].to_pickle('../inputs/datasets/char_preds/e154/e154_char_preds.pkl')

## e155

## noise analysis

In [60]:
trn_df = pd.read_csv('../inputs/origin/train.csv').dropna()
display(trn_df.shape, trn_df.head())

(27480, 4)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [61]:
ckpts = []
!mkdir -p ../checkpoints/e155/best_model_only/
for fckpt in glob('../checkpoints/e155/best/*.pth'):
    ckpt = torch.load(fckpt)
    ckpts.append(ckpt)
    fout_ckpt = {}
    fout_ckpt['model_state_dict'] = ckpt['model_state_dict']
    torch.save(fout_ckpt, re.sub('best', 'best_model_only', fckpt))

In [62]:
def _mk_char_preds(offsets, preds_head, preds_tail):
    char_preds_heads, char_preds_tails = [], []
    # 最初の４つは無視
    for offset, pred_head, pred_tail in tqdm(zip(offsets, preds_head, preds_tail)):
        char_preds_head, char_preds_tail = np.zeros(141), np.zeros(141)
        for offset_i, pred_head_i, pred_tail_i in zip(offset[4:], pred_head[4:], pred_tail[4:]):
            char_preds_head[offset_i[0]:offset_i[1]] = pred_head_i
            char_preds_tail[offset_i[0]:offset_i[1]] = pred_tail_i
        char_preds_heads.append(char_preds_head)
        char_preds_tails.append(char_preds_tail)
    char_preds_heads, char_preds_tails = np.asarray(char_preds_heads), np.asarray(char_preds_tails)
    return char_preds_heads, char_preds_tails

In [63]:
ckpts[0].keys()

dict_keys(['fold_num', 'current_epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'val_textIDs', 'val_input_ids', 'val_preds', 'val_labels', 'val_offsets', 'histories'])

In [64]:
char_preds = []

for ckpt in ckpts:
    trn_df = trn_df.set_index('textID')
    val_char_preds_heads, val_char_preds_tails = _mk_char_preds(ckpt['val_offsets'], ckpt['val_preds'][0],  ckpt['val_preds'][1])
    for textID, val_char_preds_head, val_char_preds_tail in zip(ckpt['val_textIDs'], val_char_preds_heads, val_char_preds_tails):
        char_preds.append({'textID': textID, 'char_pred_start': val_char_preds_head, 'char_pred_end': val_char_preds_tail})
    # trn_df.loc[ckpt['val_textIDs'], 'val_char_pred_head'] = val_char_preds_heads
    # trn_df.loc[ckpt['val_textIDs'], 'val_char_pred_tail'] = val_char_preds_tails
    trn_df = trn_df.reset_index()
trn_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [65]:
char_preds_df = pd.DataFrame(char_preds)
char_preds_df.head()

,textID,char_pred_start,char_pred_end
0,2d43263be0,"[0.89841628074646, 0.89841628074646, 0.8984162...","[0.0003774258366320282, 0.0003774258366320282,..."
1,c554ea2a90,"[0.03897930309176445, 0.03897930309176445, 0.0...","[0.0004395379510242492, 0.0004395379510242492,..."
2,ab039258bd,"[0.04385346546769142, 0.04385346546769142, 0.0...","[0.00030158201116137207, 0.0003015820111613720..."
3,5a85ab5cf9,"[0.9975095987319946, 0.9975095987319946, 0.997...","[0.0005380607326515019, 0.0005380607326515019,..."
4,36f947146a,"[0.10469190776348114, 0.10469190776348114, 0.1...","[0.002065591514110565, 0.002065591514110565, 0..."


In [66]:
trn_df = trn_df.merge(char_preds_df, on='textID', how='left')
trn_df

,textID,text,selected_text,sentiment,char_pred_start,char_pred_end
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"[0.9982798099517822, 0.9982798099517822, 0.000...","[0.00024153896083589643, 0.0002415389608358964..."
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[0.3679257035255432, 0.3679257035255432, 0.367...","[0.0004281910660210997, 0.0004281910660210997,..."
2,088c60f138,my boss is bullying me...,bullying me,negative,"[0.3835388720035553, 0.3835388720035553, 0.383...","[0.0008378780912607908, 0.0008378780912607908,..."
3,9642c003ef,what interview! leave me alone,leave me alone,negative,"[0.12908340990543365, 0.12908340990543365, 0.1...","[0.0006538929301314056, 0.0006538929301314056,..."
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[0.544698178768158, 0.544698178768158, 0.54469...","[0.0010821548057720065, 0.0010821548057720065,..."
...,...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,"[0.051752399653196335, 0.051752399653196335, 0...","[0.0006701280362904072, 0.0006701280362904072,..."
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,"[0.3703768849372864, 0.3703768849372864, 0.000...","[0.0004222270508762449, 0.0004222270508762449,..."
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,"[0.3994556963443756, 0.3994556963443756, 0.002...","[0.0005538389668799937, 0.0005538389668799937,..."
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,"[0.12966763973236084, 0.12966763973236084, 0.1...","[0.0003283437108621001, 0.0003283437108621001,..."


In [67]:
trn_df.isnull().mean()

textID             0.0
text               0.0
selected_text      0.0
sentiment          0.0
char_pred_start    0.0
char_pred_end      0.0
dtype: float64

In [68]:
!mkdir -p ../inputs/datasets/char_preds/e155

In [69]:
trn_df[['textID', 'char_pred_start', 'char_pred_end']].to_pickle('../inputs/datasets/char_preds/e155/e155_char_preds.pkl')